In [14]:
# Imports
import os
import getpass

import numpy

import nipype
import nibabel
import jinja2

import nipype.interfaces.io as nio


from nipype.interfaces.base import BaseInterface
from nipype.interfaces.utility import IdentityInterface

from nipype.interfaces.base import BaseInterfaceInputSpec, File, TraitedSpec, traits
from nipype import Node, Function, Workflow
from nipype.interfaces.io import BIDSDataGrabber
from nilearn.plotting import plot_roi
from nilearn import image
from bids.layout import BIDSLayout
from nipype.pipeline import engine as pe

from matplotlib import pyplot as plt
from os.path import join
from bids.tests import get_test_data_path

In [15]:
user = getpass.getuser()

if user == 'ari':
    bids_ds_path = '/Volumes/SSD-Bercows/Joan/research/0_data/Sample_Multi-Modal_BIDS_dataset/ds-sample/'

if user == 'andres':
    bids_ds_path = '/home/andres/Desktop/BIDS/DS001_BIDS_andres/'

deriv_cmp_dir = join(bids_ds_path, "derivatives", "cmp")
deriv_nipype_dir = join(bids_ds_path, "nipype", "sub-01", "ses-01")

In [ ]:
class OverlayT1wSegmentationInputSpec(BaseInterfaceInputSpec):
    
    CSF_file = traits.List(exists=True, mandatory=True, desc='CSF file') # Do not set exists=True !!
    WM_file = traits.List(exists=True, mandatory=True, desc='WM file')
    GM_file = traits.List(exists=True, mandatory=True, desc='GM file')
    brain_file = traits.List(exists=True, mandatory=True, desc='brain file')
    t1w_file = traits.List(exists=True, mandatory=True, desc='t1w file')

    
class OverlayT1wSegmentationOutputSpec(TraitedSpec):
    fig_file = traits.List(desc='png overlay')
    
    

class OverlayT1wSegmentation(BaseInterface):
    input_spec = OverlayT1wSegmentationInputSpec
    output_spec = OverlayT1wSegmentationOutputSpec
    
    def _run_interface(self, runtime):

        # Call our python code here:
        self.fig_file = self._overlay_t1w_segmentation(
            self.inputs.t1w_file,
            self.inputs.GM_file,
            self.inputs.WM_file,
            self.inputs.CSF_file,
            self.inputs.brain_file
        )

        # And we are done
        return runtime
    
    def _overlay_t1w_segmentation(self, t1w_path, GM_path, WM_path, CSF_path, brain_path):
        
        t1w_image = nibabel.load(t1w_path[0])
        image_list = [nibabel.load(GM_path[0]), 
                      nibabel.load(WM_path[0]), 
                      nibabel.load(CSF_path[0]),
                      nibabel.load(brain_path[0])]
        
        tissues = ['GM', 'WM', 'CSF', 'brain']
        output_files = []
        for i,t in enumerate(tissues):
            output_file = bids_ds_path + 'derivatives/nipype/sub-01/'+t+'_overlay.png'
            fig_file = plot_roi(image_list[i], t1w_image, output_file = output_file)
            output_files.append(output_file)
        
        return output_files
    
    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['fig_file'] = self.fig_file
        return outputs

In [41]:
class ReportInputSpec(BaseInterfaceInputSpec):
    fig_file = traits.List(desc='png overlay')

    
class ReportOutputSpec(TraitedSpec):
    report_file = File(desc='HTML report file')
    

class Report(BaseInterface):
    input_spec = ReportInputSpec
    #output_spec = ReportOutputSpec
    
    def _run_interface(self, runtime):
        
        # Call our python code here:
        self.fig_file = self._generate_html(
            self.inputs.fig_file
        )
        
        # And we are done
        return runtime
    
    def _generate_html(self, fig_file):
        title = 'Hi this is my title'
        outputfile = '/home/andres/Desktop/BIDS/automatedHTML/report_segmentation.html'
        
        my_images = []
        for image in fig_file:
            my_images.append([image, image.split('/')[-1].split('.')[-2]])
            
        subs = jinja2.Environment(
                      loader=jinja2.FileSystemLoader('/home/andres/Desktop/BIDS/automatedHTML/templates')
                      ).get_template('template_segmentation.html').render(title=title,mydata=my_images)
        # lets write the substitution to a file
        with open(outputfile,'w') as f: f.write(subs)

In [42]:
bg = Node(BIDSDataGrabber(index_derivatives=True), name='bids-grabber')
bg.inputs.base_dir = bids_ds_path

bg.inputs.subject = '01'

bg.inputs.output_query = {'WM': {'scope': 'Connectome Mapper',
                                  'space': None,
                                  'desc': None,
                                  'sufix': 'dseg',
                                  'label': 'WM',
                                  'extensions': ['nii.gz']},
                          'GM': {'scope': 'Connectome Mapper',
                                  'space': None,
                                  'sufix': 'dseg',
                                  'label': 'GM',
                                  'extensions': ['nii.gz']},
                          'CSF': {'scope': 'Connectome Mapper',
                                  'space': None,
                                  'desc': None,
                                  'sufix': 'dseg',
                                  'label': 'CSF',
                                  'extensions': ['nii.gz']},
                          'brain': {'scope': 'Connectome Mapper',
                                  'sufix': 'dseg',
                                  'label': 'brain',
                                  'extensions': ['nii.gz']},
                          't1w': {'scope': 'Connectome Mapper',
                                  'suffix': 'T1w',
                                  'extensions': ['nii.gz']}
                         }

In [43]:
overlay = Node(OverlayT1wSegmentation(), name='overlay_fig')
report = Node(Report(), name='report')
datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory = deriv_nipype_dir

In [44]:
workflow = Workflow("overlay_workflow")

workflow.connect([(bg, overlay,
                   [('CSF', 'CSF_file'),
                    ('GM', 'GM_file'),
                    ('WM', 'WM_file'),
                    ('brain', 'brain_file'),
                    ('t1w','t1w_file')]
                  )])
workflow.connect(overlay, 'fig_file', datasink, 'fig_input')
workflow.connect(overlay, 'fig_file', report, 'fig_file')

workflow.run()

210321-17:23:39,499 nipype.workflow INFO:
	 Workflow overlay_workflow settings: ['check', 'execution', 'logging', 'monitoring']
210321-17:23:39,508 nipype.workflow INFO:
	 Running serially.
210321-17:23:39,509 nipype.workflow INFO:
	 [Node] Setting-up "overlay_workflow.bids-grabber" in "/tmp/tmp01qejuvu/overlay_workflow/bids-grabber".
210321-17:23:39,517 nipype.workflow INFO:
	 [Node] Running "bids-grabber" ("nipype.interfaces.io.BIDSDataGrabber")
210321-17:23:40,208 nipype.workflow INFO:
	 [Node] Finished "overlay_workflow.bids-grabber".
210321-17:23:40,211 nipype.workflow INFO:
	 [Node] Setting-up "overlay_workflow.overlay_fig" in "/tmp/tmp_yngcozr/overlay_workflow/overlay_fig".
210321-17:23:40,216 nipype.workflow INFO:
	 [Node] Running "overlay_fig" ("__main__.OverlayT1wSegmentation")
210321-17:23:46,881 nipype.workflow INFO:
	 [Node] Finished "overlay_workflow.overlay_fig".
210321-17:23:46,882 nipype.workflow INFO:
	 [Node] Setting-up "overlay_workflow.report" in "/tmp/tmp7n6wbmk7/